# The Dwarves of Wroclav

Have you ever been to Wroclav? It's a cute little University city on the Oder river in western Poland. It is famous for being the historical capital of Silesia and for the Centennial Hall, the Unesco World Heritage Site designed by Max Berg in 1911 - 1913. However, in my opinion one of the most interesting quality of the city is that among many striking architectural curiosity, musea and beautiful historical neigborhoods it hides 350 statues of little dwarves, commemorating a shop or a building or just hanging out at some street corner. 

I went there in November 2018 with a companion and of course Dwarf-spotting quickly became our main activity. The person to spot the most dwarves would win a pizza with beer from the best pizza place in Munich. Obviously I won, with a crushing 144 dwarves-sightings vs. the 96 of my companion.

However, we were never able to find a satisfying map of the little dwarves. Thus, when we came home I decided to code one using Folium and the GPS coordinates found on Polish Wikipedia. Use it wisely!


In [ ]:
import re
import sys
import csv
import time
import base64
import folium
import logging
import requests
import numpy as np
import pandas as pd
import unicodedata
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
#from mpl_toolkits.basemap import Basemap
from requests.exceptions import ConnectionError, RequestException

In [6]:
# Polish wikipedia page on the dwarves of Wreslaw
wikiurl = requests.get('https://pl.wikipedia.org/wiki/Wroc%C5%82awskie_krasnale').text
# Acces the URL using BeautifulSoup
soup = BeautifulSoup(wikiurl, 'lxml')

# Pick the right table in the article
TableImg = soup.find_all("table", class_="wikitable sortable")
neededT = 0
for ind,T in enumerate(TableImg): 
    if T.find("span", class_="coordinates inline plainlinks"): 
        neededT = ind

# Wrapper
Wrapper = TableImg[neededT].find_all("tr")

# Retrieve the dwarves coordinates
# lat and lon have the coordinates both in degrees and sexagesimal
# we need to retrieve only the coordinates in degrees.
lat = []
lon = []
indexC = []
for i in range(len(Wrapper)): 
    try: 
        lat.append(Wrapper[i].find_all("span", class_='latitude')[-1].getText())
        lon.append(Wrapper[i].find_all("span", class_='longitude')[-1].getText())
    except IndexError: 
        indexC.append(i)        

lat = np.array([lat[i].replace(',','.') for i in range(len(lat))])
lon = np.array([lon[i].replace(',','.') for i in range(len(lat))])

Wrapper = np.delete(Wrapper, indexC)

#Wraplat = soup.find_all('span', class_="latitude")                                                                                                              
#Wraplong = soup.find_all('span', class_="longitude") 

# Get images and dimesions
# I will link the image of each dwarf to the coordinate point on the map.
img = []
imgW = []
imgH = []
index = []
for i in range(len(Wrapper)): 
    try: 
        img.append(Wrapper[i].find("a", class_='image').find("img")["src"]) 
        imgW.append(Wrapper[i].find("a", class_='image').find("img")["width"])
        imgH.append(Wrapper[i].find("a", class_='image').find("img")["height"])
    except AttributeError: 
        index.append(i)        


imgW = np.array(imgW).astype(np.int)
imgH = np.array(imgH).astype(np.int)
img = np.array([img[i].replace('{}px'.format(imgW[i]),'{}px'.format(imgW[i]*2)) for i in range(len(img))])


lat_deg = np.delete(lat,np.array(index)).astype(np.float)
lon_deg = np.delete(lon,np.array(index)).astype(np.float)


iframe = []
for i,Dw in enumerate(img):
    html = '<img src="https:{}" style="width:{}px;height:{}px;">'.format
    iframe.append(folium.IFrame(html(Dw,imgW[i]*2.2, imgH[i]*2.2), 
                  width=imgW[i]*2.2+20, height=imgH[i]*2.2+20))

popup = []
for i in range(len(iframe)):
    popup.append(folium.Popup(iframe[i], max_width=2650))

# The map:
folium_map = folium.Map(location=[51.1079, 17.0385],
                        zoom_start=12,
                        tiles= "Stamen Terrain") #"CartoDB dark_matter")

# The Dwarves:
for i in range(len(lat_deg)):
    folium.CircleMarker(location=[lat_deg[i], lon_deg[i]], radius=4, 
                        color='None', fill_color='#FF4633',fill_opacity=0.5,popup=popup[i]).add_to(folium_map)

folium_map
folium_map.save("wroslaw.html")